In [271]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression, LinearSVC
import pyspark.ml.tuning as tune
import pyspark.ml.evaluation as evals

from pyspark.sql import SparkSession
import pyspark.sql.functions as fn

# Any results you write to the current directory are saved as output.

['bigml_59c28831336c6604c800002a.csv']


In [217]:
spark = SparkSession.builder.appName('telecom').getOrCreate()

In [218]:
telecom = spark.read.csv('../input/bigml_59c28831336c6604c800002a.csv', header=True)

In [219]:
telecom.printSchema()

root
 |-- state: string (nullable = true)
 |-- account length: string (nullable = true)
 |-- area code: string (nullable = true)
 |-- phone number: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- number vmail messages: string (nullable = true)
 |-- total day minutes: string (nullable = true)
 |-- total day calls: string (nullable = true)
 |-- total day charge: string (nullable = true)
 |-- total eve minutes: string (nullable = true)
 |-- total eve calls: string (nullable = true)
 |-- total eve charge: string (nullable = true)
 |-- total night minutes: string (nullable = true)
 |-- total night calls: string (nullable = true)
 |-- total night charge: string (nullable = true)
 |-- total intl minutes: string (nullable = true)
 |-- total intl calls: string (nullable = true)
 |-- total intl charge: string (nullable = true)
 |-- customer service calls: string (nullable = true)
 |-- churn: string (nullable = true)



In [220]:
telecom.count()

3333

In [221]:
def count_missing(df):
    return df.agg(
        *[(1 - fn.count(c)/fn.count('*')).alias(c+'_missing') for c in df.columns]
    )

In [222]:
# count ratio of missing value at each column
count_missing(telecom).show()

+-------------+----------------------+-----------------+--------------------+--------------------------+-----------------------+-----------------------------+-------------------------+-----------------------+------------------------+-------------------------+-----------------------+------------------------+---------------------------+-------------------------+--------------------------+--------------------------+------------------------+-------------------------+------------------------------+-------------+
|state_missing|account length_missing|area code_missing|phone number_missing|international plan_missing|voice mail plan_missing|number vmail messages_missing|total day minutes_missing|total day calls_missing|total day charge_missing|total eve minutes_missing|total eve calls_missing|total eve charge_missing|total night minutes_missing|total night calls_missing|total night charge_missing|total intl minutes_missing|total intl calls_missing|total intl charge_missing|customer service cal

In [223]:
# convert some sample to pandas df to see some value
def to_pandas_sample(df):
    pd_sample = df.sample(True, 0.01, seed=43).toPandas()
    return pd_sample

In [224]:
to_pandas_sample(telecom).head(1)

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,ID,119,415,398-1294,no,no,0,159.1,114,27.05,231.3,117,19.66,143.2,91,6.44,8.8,3,2.38,5,True


In [225]:
# check if international plan and voice mail plan only contains yes/no value
u1 = telecom.select('international plan').distinct().count()
u2 = telecom.select('voice mail plan').distinct().count()

print(u1)
print(u2)

2
2


In [226]:
to_pandas_sample(telecom).head(1)

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,ID,119,415,398-1294,no,no,0,159.1,114,27.05,231.3,117,19.66,143.2,91,6.44,8.8,3,2.38,5,True


In [227]:
# convert numeric data to integer or double
int_cols = ['account length', 'area code', 'number vmail messages', 
            'total day calls', 'total eve calls', 'total night calls',
            'total intl calls', 'customer service calls']

double_cols = ['total day minutes', 'total day charge', 'total eve minutes', 'total eve charge', 'total night minutes',
              'total night charge', 'total intl minutes','total intl charge']

def cast_data_type(df, list_cols, data_type='double'):
    for col in list_cols:
        df = df.withColumn(col, df[col].cast(data_type))
    return df

In [228]:
telecom_casted = cast_data_type(telecom, int_cols, 'integer')
telecom_casted = cast_data_type(telecom_casted, double_cols, 'double')

In [229]:
telecom_casted.printSchema()

root
 |-- state: string (nullable = true)
 |-- account length: integer (nullable = true)
 |-- area code: integer (nullable = true)
 |-- phone number: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- number vmail messages: integer (nullable = true)
 |-- total day minutes: double (nullable = true)
 |-- total day calls: integer (nullable = true)
 |-- total day charge: double (nullable = true)
 |-- total eve minutes: double (nullable = true)
 |-- total eve calls: integer (nullable = true)
 |-- total eve charge: double (nullable = true)
 |-- total night minutes: double (nullable = true)
 |-- total night calls: integer (nullable = true)
 |-- total night charge: double (nullable = true)
 |-- total intl minutes: double (nullable = true)
 |-- total intl calls: integer (nullable = true)
 |-- total intl charge: double (nullable = true)
 |-- customer service calls: integer (nullable = true)
 |-- churn: string (nullable =

In [230]:
# one-hot encoder categorical data
stages = []

cate_cols = ['state', 'international plan','voice mail plan']
for col in cate_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col+'_index')
    encoder = OneHotEncoder(inputCol=col+'_index', outputCol=col+'_fact')
    stages.append(indexer)
    #stages.append(encoder)

In [231]:
# assemble to 1 column feature
feature_cols = ['state_index', 'account length', 'area code', 'international plan_index', 'voice mail plan_index', 
                'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 
                'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 
                'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 
                'total intl charge', 'customer service calls']

print(feature_cols)

['state_fact', 'account length', 'area code', 'international plan_fact', 'voice mail plan_fact', 'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls']


In [232]:
vec_assemble = VectorAssembler(inputCols=feature_cols, outputCol='features')
stages.append(vec_assemble)

In [233]:
def boolean_to_int(x):
    return 1 if x == 'True' else 0

In [234]:
boolean_to_int_udf = fn.udf(boolean_to_int, 'integer')

In [235]:
# convert churn label to 0 and 1
telecom_casted = telecom_casted.withColumn('churn', boolean_to_int_udf('churn'))

In [236]:
telecom_casted = telecom_casted.withColumnRenamed('churn', 'label')

In [237]:
telecom_casted.printSchema()

root
 |-- state: string (nullable = true)
 |-- account length: integer (nullable = true)
 |-- area code: integer (nullable = true)
 |-- phone number: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- number vmail messages: integer (nullable = true)
 |-- total day minutes: double (nullable = true)
 |-- total day calls: integer (nullable = true)
 |-- total day charge: double (nullable = true)
 |-- total eve minutes: double (nullable = true)
 |-- total eve calls: integer (nullable = true)
 |-- total eve charge: double (nullable = true)
 |-- total night minutes: double (nullable = true)
 |-- total night calls: integer (nullable = true)
 |-- total night charge: double (nullable = true)
 |-- total intl minutes: double (nullable = true)
 |-- total intl calls: integer (nullable = true)
 |-- total intl charge: double (nullable = true)
 |-- customer service calls: integer (nullable = true)
 |-- label: integer (nullable 

In [238]:
# apply all transformation to data
pipeline = Pipeline(stages=stages)
pipe_data = pipeline.fit(telecom_casted).transform(telecom_casted)

In [239]:
# split train and test
train, test = pipe_data.randomSplit([.7, .3])

In [240]:
# logistic
logistic = LogisticRegression()

In [241]:
# Evaluation metrics
evaluator = evals.BinaryClassificationEvaluator(rawPredictionCol='probability')

In [242]:
# tuner
grid = tune.ParamGridBuilder().addGrid(logistic.maxIter, [2,10,50]).addGrid(logistic.regParam, [0.01,0.05,0.3]).build()
                                    

In [243]:
# cross validator
cv = tune.CrossValidator(estimator=logistic,
                        estimatorParamMaps=grid,
                        evaluator=evaluator,
                        numFolds=7)

In [244]:
# fit to cross validator
cv_Models = cv.fit(train)


In [245]:
best_model = cv_Models.bestModel

In [246]:
acc_score = best_model.evaluate(dataset=train).accuracy
acc_score

0.862937660119556

In [247]:
pred = best_model.transform(dataset=train)

In [248]:
pred.printSchema()

root
 |-- state: string (nullable = true)
 |-- account length: integer (nullable = true)
 |-- area code: integer (nullable = true)
 |-- phone number: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- number vmail messages: integer (nullable = true)
 |-- total day minutes: double (nullable = true)
 |-- total day calls: integer (nullable = true)
 |-- total day charge: double (nullable = true)
 |-- total eve minutes: double (nullable = true)
 |-- total eve calls: integer (nullable = true)
 |-- total eve charge: double (nullable = true)
 |-- total night minutes: double (nullable = true)
 |-- total night calls: integer (nullable = true)
 |-- total night charge: double (nullable = true)
 |-- total intl minutes: double (nullable = true)
 |-- total intl calls: integer (nullable = true)
 |-- total intl charge: double (nullable = true)
 |-- customer service calls: integer (nullable = true)
 |-- label: integer (nullable 

In [249]:
pred.filter(fn.col('label')==1).select(*['rawPrediction', 'probability', 'label', 'prediction']).sample(False, 0.1).show()

+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|label|prediction|
+--------------------+--------------------+-----+----------+
|[2.01691914104582...|[0.88256206576326...|    1|       0.0|
|[0.24397917888983...|[0.56069402111125...|    1|       0.0|
|[0.79761917323349...|[0.68946496898672...|    1|       0.0|
|[1.52574961884067...|[0.82138358136222...|    1|       0.0|
|[-0.3919519133808...|[0.40324750573448...|    1|       1.0|
|[1.06448651218027...|[0.74354698964691...|    1|       0.0|
|[1.12792261679826...|[0.75545532165957...|    1|       0.0|
|[0.98261736650703...|[0.72762725024344...|    1|       0.0|
|[0.97639958814346...|[0.72639323207253...|    1|       0.0|
|[0.92172519242516...|[0.71539349510022...|    1|       0.0|
|[1.04574015431237...|[0.73995605326374...|    1|       0.0|
|[-0.3183676294360...|[0.42107362049739...|    1|       1.0|
|[0.96314122087336...|[0.72375028887097...|    1|       0.0|
|[1.61434920869665...|[0

In [250]:
pred.filter(fn.col('label')==1).count()

346

In [251]:
pred.filter(fn.col('label')==0).count()

1996

In [255]:
dt = DecisionTreeClassifier()
dt_model = dt.fit(train)

# grid = tune.ParamGridBuilder().addGrid(dt.max)

In [259]:
pred_dt = dt_model.transform(train)
score = evaluator.evaluate(pred_dt)

In [260]:
score

0.9232974619759751

In [261]:
pred_dt.filter(fn.col('label')==1).select(*['rawPrediction', 'probability', 'label', 'prediction']).sample(False, 0.1).show()

+-------------+--------------------+-----+----------+
|rawPrediction|         probability|label|prediction|
+-------------+--------------------+-----+----------+
|   [5.0,62.0]|[0.07462686567164...|    1|       1.0|
|   [5.0,62.0]|[0.07462686567164...|    1|       1.0|
|   [1.0,10.0]|[0.09090909090909...|    1|       1.0|
|   [0.0,41.0]|           [0.0,1.0]|    1|       1.0|
|   [0.0,41.0]|           [0.0,1.0]|    1|       1.0|
|    [1.0,2.0]|[0.33333333333333...|    1|       1.0|
|[1363.0,30.0]|[0.97846374730796...|    1|       0.0|
|   [0.0,28.0]|           [0.0,1.0]|    1|       1.0|
|   [0.0,28.0]|           [0.0,1.0]|    1|       1.0|
|   [5.0,62.0]|[0.07462686567164...|    1|       1.0|
|  [109.0,8.0]|[0.93162393162393...|    1|       0.0|
|  [136.0,9.0]|[0.93793103448275...|    1|       0.0|
|   [5.0,62.0]|[0.07462686567164...|    1|       1.0|
|   [5.0,62.0]|[0.07462686567164...|    1|       1.0|
|[1363.0,30.0]|[0.97846374730796...|    1|       0.0|
|   [0.0,59.0]|           [0

In [262]:
# test score
test_score = evaluator.evaluate(dt_model.transform(test))

In [263]:
test_score

0.866903707755688

In [269]:
def get_accuracy(df):
    true_pred = dt_model.transform(df).filter(fn.col('label')==fn.col('prediction')).count()
    all_pred = dt_model.transform(df).count()
    return true_pred/all_pred

In [270]:
print("decision tree train accuracy: ", get_accuracy(train))
print("decision tree test accuracy: ", get_accuracy(test))

decision tree train accuracy:  0.9470538001707942
decision tree test accuracy:  0.9323915237134208
